In [37]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neural_network import MLPClassifier
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, RNN, LSTMCell
from keras import layers
from keras import optimizers
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


def createXorData(samples=100000, size=50):
  """
  generates a dataset for the XOR problem in 
    https://blog.openai.com/requests-for-research-2/
  LHS are sequences of bools
  RHS computes the parity bit 
    (eg. wether the sequence has odd or even sum)
  """
  lhs = np.zeros((samples, size), dtype=np.float32)
  rhs = np.zeros(samples)
  for row in range(samples):
    lhs[row] = np.random.randint(0, 2, size)
    rhs[row] = lhs[row].sum() % 2
  return lhs, rhs

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12986928932043762479
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 338690048
locality {
  bus_id: 1
}
incarnation: 15964170320547248916
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [0]:
SIZE = 4
SAMPLES = 64000
BATCH_SIZE = 32

Xtrain, Ytrain = createXorData(SAMPLES, SIZE)
Xtest, Ytest = createXorData(SAMPLES//10, SIZE)

In [76]:
"""
Training data is shaped as a normal matrix
[[0,1,1], [1,0,0], ..., [1,1,1]]

The RNN cell takes as input a sequence of sequences,
so to have a rnn sequence length of [# features] 
the data needs to be reshaped as
[[[0],[1],[1]], [[1],[0],[0]], ..., [[1],[1],[1]]]

(alternatively the data can be further split 
 in more sub-array of sequences in the features)
"""
model = Sequential()
model.add(layers.LSTM(50, batch_input_shape=(BATCH_SIZE, SIZE, 1), 
               kernel_initializer='random_uniform', 
               stateful=True))
model.add(Dense(1, activation='sigmoid'))

model_opt = optimizers.Adadelta(lr=0.5, rho=0.95, epsilon=None, decay=0.0)
model.compile(optimizer=model_opt, loss='binary_crossentropy', metrics=['acc'])

model.summary()
history = model.fit(
    # reshape training data in 
    Xtrain.reshape((len(Xtrain), SIZE, 1)),
    Ytrain, epochs=350,
    batch_size=BATCH_SIZE,
    validation_split=0.1, 
    shuffle=False, verbose=False)
#
# Plotting output
#
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.figure()

acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
preds = model.predict(Xtest.reshape((len(Xtest), SIZE, 1)), 
                      batch_size=BATCH_SIZE)
preds = np.round(preds[:, 0]).astype('float32')
acc = (np.sum(preds == Ytest) / len(Ytest)) * 100.
print('Accuracy: {:.2f}%'.format(acc))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (32, 50)                  10400     
_________________________________________________________________
dense_38 (Dense)             (32, 1)                   51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


KeyboardInterrupt: ignored

In [0]:
# This returns a tensor
inputs = layers.Input(batch_shape=(BATCH_SIZE, SIZE, 1))

cell_size = 50

cells = [
    LSTMCell(cell_size)
]
input_cells = keras.layers.StackedRNNCells(cells)

x = keras.layers.RNN(input_cells, stateful=True)(inputs)
logits = Dense(1, activation='sigmoid')(x)

model = keras.models.Model(inputs=inputs, outputs=logits)
model_opt = optimizers.Adadelta(lr=0.5, rho=0.95, epsilon=None, decay=0.0)
model.compile(optimizer=model_opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()
model.fit(Xtrain.reshape((len(Xtrain), SIZE, 1)),
          Ytrain, batch_size=BATCH_SIZE, epochs=350)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        (32, 4, 1)                0         
_________________________________________________________________
rnn_28 (RNN)                 (32, 50)                  10400     
_________________________________________________________________
dense_41 (Dense)             (32, 1)                   51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________
Epoch 1/350
18784/64000 [=======>......................] - ETA: 22s - loss: 0.6934 - acc: 0.4950

64000/64000 [==============================] - 25s 386us/step - loss: 0.6934 - acc: 0.4931
Epoch 2/350
64000/64000 [==============================] - 22s 337us/step - loss: 0.6932 - acc: 0.5052
Epoch 3/350
  352/64000 [..............................] - ETA: 23s - loss: 0.6932 - acc: 0.4972

64000/64000 [==============================] - 21s 335us/step - loss: 0.6933 - acc: 0.5021
Epoch 4/350
56512/64000 [=========================>....] - ETA: 2s - loss: 0.6932 - acc: 0.5043

64000/64000 [==============================] - 22s 336us/step - loss: 0.6933 - acc: 0.5036
Epoch 5/350
64000/64000 [==============================] - 22s 339us/step - loss: 0.6932 - acc: 0.5010
Epoch 6/350
11008/64000 [====>.........................] - ETA: 17s - loss: 0.6931 - acc: 0.4996

64000/64000 [==============================] - 21s 336us/step - loss: 0.6932 - acc: 0.5006
Epoch 7/350
61696/64000 [===========================>..] - ETA: 0s - loss: 0.6932 - acc: 0.5002

...

64000/64000 [==============================] - 21s 333us/step - loss: 0.6932 - acc: 0.5035
Epoch 10/350
61984/64000 [============================>.] - ETA: 0s - loss: 0.6932 - acc: 0.4992

64000/64000 [==============================] - 21s 334us/step - loss: 0.6932 - acc: 0.4992
Epoch 11/350
64000/64000 [==============================] - 21s 332us/step - loss: 0.6932 - acc: 0.5011
Epoch 12/350
12704/64000 [====>.........................] - ETA: 17s - loss: 0.6931 - acc: 0.5076

64000/64000 [==============================] - 21s 333us/step - loss: 0.6932 - acc: 0.5015
Epoch 13/350
62080/64000 [============================>.] - ETA: 0s - loss: 0.6932 - acc: 0.5009

64000/64000 [==============================] - 21s 330us/step - loss: 0.6932 - acc: 0.5013
Epoch 14/350
64000/64000 [==============================] - 21s 330us/step - loss: 0.6932 - acc: 0.5016
Epoch 15/350
12800/64000 [=====>........................] - ETA: 17s - loss: 0.6932 - acc: 0.5034

64000/64000 [==============================] - 21s 332us/step - loss: 0.6932 - acc: 0.5005
Epoch 16/350
26848/64000 [===========>..................] - ETA: 12s - loss: 0.6932 - acc: 0.4988

In [3]:
#
# overkill MLP
#
mlp = MLPClassifier(hidden_layer_sizes=[10, 10, 10],
                    activation="relu"
                   ).fit(Xtrain, Ytrain)
pred_score = mlp.score(Xtest, Ytest)
print(pred_score)

1.0


In [14]:
from copy import deepcopy
results_list = []
mlp_layers = [2]
for input_size in range(1, 24):
  print("\nLen = ", input_size)
  Xtrain, Ytrain = createXorData(50000, input_size)
  Xtest, Ytest = createXorData(5000, input_size)
  pred_score = 0.0
  while pred_score < 0.85:
    print("Trying ", mlp_layers)
    mlp = MLPClassifier(hidden_layer_sizes=mlp_layers,
                      activation="relu"
                     ).fit(Xtrain, Ytrain)
    pred_score = mlp.score(Xtest, Ytest)
    if pred_score < 0.85:
      # If huge number of neurons, add a layer
      if mlp_layers[0] >= 50:
        mlp_layers.append(mlp_layers[0])
      mlp_layers = [i * 2 for i in mlp_layers]
  results_list.append(deepcopy(mlp_layers))
  print(results_list)


Len =  1
Trying  [2]
Trying  [4]
[[4]]

Len =  2
Trying  [4]
[[4], [4]]

Len =  3
Trying  [4]
[[4], [4], [4]]

Len =  4
Trying  [4]
Trying  [8]
[[4], [4], [4], [8]]

Len =  5
Trying  [8]
[[4], [4], [4], [8], [8]]

Len =  6
Trying  [8]
Trying  [16]
[[4], [4], [4], [8], [8], [16]]

Len =  7
Trying  [16]
[[4], [4], [4], [8], [8], [16], [16]]

Len =  8
Trying  [16]
[[4], [4], [4], [8], [8], [16], [16], [16]]

Len =  9
Trying  [16]


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[[4], [4], [4], [8], [8], [16], [16], [16], [16]]

Len =  10
Trying  [16]
Trying  [32]
[[4], [4], [4], [8], [8], [16], [16], [16], [16], [32]]

Len =  11
Trying  [32]
[[4], [4], [4], [8], [8], [16], [16], [16], [16], [32], [32]]

Len =  12
Trying  [32]
[[4], [4], [4], [8], [8], [16], [16], [16], [16], [32], [32], [32]]

Len =  13
Trying  [32]
[[4], [4], [4], [8], [8], [16], [16], [16], [16], [32], [32], [32], [32]]

Len =  14
Trying  [32]
[[4], [4], [4], [8], [8], [16], [16], [16], [16], [32], [32], [32], [32], [32]]

Len =  15
Trying  [32]
[[4], [4], [4], [8], [8], [16], [16], [16], [16], [32], [32], [32], [32], [32], [32]]

Len =  16
Trying  [32]
[[4], [4], [4], [8], [8], [16], [16], [16], [16], [32], [32], [32], [32], [32], [32], [32]]

Len =  17
Trying  [32]
[[4], [4], [4], [8], [8], [16], [16], [16], [16], [32], [32], [32], [32], [32], [32], [32], [32]]

Len =  18
Trying  [32]
[[4], [4], [4], [8], [8], [16], [16], [16], [16], [32], [32], [32], [32], [32], [32], [32], [32], [32]]



In [0]:
[[4], [4], [4], [8], [8], [16], [16], [16], [16], [32], [32], [32], [32], [32], [32], [32], [32], [32], [32], [32], [32], [64], [64]]